In [ ]:
# Panel Dashboard with ipyleaflet (requires kernel)
import contextlib
import json
from pathlib import Path

import panel as pn
from ipyleaflet import Map, GeoJSON

with contextlib.suppress(Exception):
    from hefs_fews_hub.dashboard_funcs import (
        create_start_standalone_command,
        write_shell_file,
        s3_download_file,
        write_fews_desktop_shortcut,
        s3_download_directory_cli,
        set_up_logger,
        install,
        install_fews_standalone
    )

pn.extension("ipywidgets", sizing_mode="stretch_width")

In [ ]:
ACCENT_BASE_COLOR = "#5d6d7e"
RFC_BOUNDARIES = Path("geo/rfc_boundaries.geojson").resolve()
if not RFC_BOUNDARIES.exists():
    import hefs_fews_hub
    RFC_BOUNDARIES = Path(hefs_fews_hub.__file__).parent / "geo" / "rfc_boundaries.geojson"

FEWS_INSTALL_DIR = Path("/opt", "fews")
MAP_CENTER_X = 38.80
MAP_CENTER_Y = -99.14
BUCKET_NAME = "ciroh-rti-hefs-data"
RFC_IDS = [
    "ABRFC", "APRFC", "CBRFC", "CNRFC", "LMRFC", "MARFC",
    "MBRFC", "NCRFC", "NERFC", "NWRFC", "OHRFC", "SERFC", "WGRFC"
]

In [ ]:
# Helper functions
def turn_off_indeterminate():
    indeterminate.name = "Download Complete"
    indeterminate.active = False
    indeterminate.visible = False

def turn_on_indeterminate():
    indeterminate.name = "Downloading..."
    indeterminate.visible = True
    indeterminate.active = True

def on_geojson_click(event, feature, **kwargs):
    rfc_selector.value = feature['properties']["BASIN_ID"]
    
def install_fews_standalone_pf(event) -> None:
    """Download standalone configuration from S3 to the working directory."""
    turn_on_indeterminate()
    install_fews_standalone(download_dir_text.value, rfc_selector.value)
    turn_off_indeterminate()
    return

In [ ]:
# Create ipyleaflet map
with open(RFC_BOUNDARIES) as f:
    geojson_data = json.load(f)

geojson_layer = GeoJSON(
    data=geojson_data,
    hover_style={
        'color': 'red', 'dashArray': '0', 'fillOpacity': 0.6
    },
)

center = (MAP_CENTER_X, MAP_CENTER_Y)
lmap = Map(center=center, zoom=4, height=500)
lmap.add(geojson_layer)
geojson_layer.on_click(on_geojson_click)

In [ ]:
# Widgets
rfc_selector = pn.widgets.Select(name="", options=RFC_IDS, value=RFC_IDS[5])

download_dir_text = pn.widgets.TextInput(
    name='Directory to download the data:',
    value='/home/jovyan'
)

download_configs_button = pn.widgets.Button(
    name='Download Configs',
    button_type='primary'
)
download_configs_button.on_click(install_fews_standalone)

download_data_button = pn.widgets.Button(
    name='Download Data',
    button_type='primary'
)

indeterminate = pn.indicators.Progress(
    name='Indeterminate Progress',
    active=False,
    visible=False,
    styles={"height": "15px"},
)

In [ ]:
# Layout
download_row = pn.Row(
    rfc_selector,
    download_configs_button,
    download_data_button
)

column = pn.Column(
    pn.pane.IPyWidget(lmap, sizing_mode="stretch_both", min_height=500),
    download_row,
    pn.Row(download_dir_text),
    pn.Row(indeterminate)
)

In [ ]:
# Display dashboard
import hefs_fews_hub
logo_path = Path(hefs_fews_hub.__file__).parent / "images" / "CIROHLogo_200x200.png"

template = pn.template.FastListTemplate(
    site="HEFS-FEWS",
    title="Exploration System Dashboard",
    logo=str(logo_path) if logo_path.exists() else None,
    header_background=ACCENT_BASE_COLOR,
    accent_base_color=ACCENT_BASE_COLOR,
    main=[column],
)

template.servable()